In [1]:
import numpy as np
import pandas as pd 
from scipy.optimize import minimize, Bounds
# import polars as pl
# from scipy.stats import skew, kurtosis
# import warnings
# import os
# import kaggle_evaluation.default_inference_server 

# warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.tail()

,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
8985,8985,0,0,0,0,0,0,0,0,0,...,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,0.002457,0.000155,0.001990
8986,8986,0,0,0,0,0,0,0,0,0,...,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.002312,0.000156,0.001845
8987,8987,0,0,1,0,0,0,0,0,0,...,0.481481,0.787698,0.834898,0.823413,-0.723949,0.133929,-0.670946,0.002891,0.000156,0.002424
8988,8988,0,0,0,0,0,0,0,0,0,...,0.655423,0.783730,0.994026,0.851852,-0.684937,0.101852,-0.646265,0.008310,0.000156,0.007843
8989,8989,0,0,0,0,0,0,0,0,0,...,0.066799,0.783730,1.068037,0.879630,-0.764806,0.079034,-0.705662,0.000099,0.000156,-0.000368


In [3]:
df_train_opt = df_train.copy()
df_train_opt['ragged_forward_returns'] = df_train_opt['forward_returns'].shift(1)
df_train_opt['ragged_risk_free_rate'] = df_train_opt['risk_free_rate'].shift(1)
df_train_opt['predicted_return'] = df_train_opt['forward_returns']

df_train_opt = df_train_opt[['predicted_return', 'ragged_forward_returns', 'ragged_risk_free_rate', 'forward_returns', 'risk_free_rate']]

In [4]:
df_train_opt.head()

,predicted_return,ragged_forward_returns,ragged_risk_free_rate,forward_returns,risk_free_rate
0,-0.002421,NaN,NaN,-0.002421,0.000301
1,-0.008495,-0.002421,0.000301,-0.008495,0.000303
2,-0.009624,-0.008495,0.000303,-0.009624,0.000301
3,0.004662,-0.009624,0.000301,0.004662,0.000299
4,-0.011686,0.004662,0.000299,-0.011686,0.000299


In [7]:
# ポジション最適化のスクリプト

# 設定
MIN_POSITION = 0.0
MAX_POSITION = 2.0
TRADING_DAYS_PER_YEAR = 252
LOOKBACK_WINDOW = 180
EPS = 1e-10

# コンペの評価用の調整済みシャープ値の計算
def calculate_adjusted_sharpe(solution_df, predictions):
    
    solution = solution_df.copy()
    solution['position'] = predictions
    
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )
    
    strategy_excess = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_cum_excess = (1 + strategy_excess).prod()
    
    if strategy_cum_excess <= 0:
        return -1000.0
    
    strategy_mean_excess = strategy_cum_excess ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    
    if strategy_std == 0:
        return 0.0
    
    sharpe = strategy_mean_excess / strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR)
    strategy_volatility = strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100
    
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum_excess = (1 + market_excess).prod()
    
    if market_cum_excess <= 0:
        market_mean_excess = -1.0
    else:
        market_mean_excess = market_cum_excess ** (1 / len(solution)) - 1
    
    market_std = solution['forward_returns'].std()
    market_volatility = market_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100
    
    excess_vol_penalty = 1.0
    if market_volatility > 0:
        vol_ratio = strategy_volatility / market_volatility
        if vol_ratio > 1.2:
            excess_vol_penalty = 1 + (vol_ratio - 1.2)
    
    return_gap = max(0, (market_mean_excess - strategy_mean_excess) * 100 * TRADING_DAYS_PER_YEAR)
    return_penalty = 1 + (return_gap ** 2) / 100
    
    adjusted_sharpe = sharpe / (excess_vol_penalty * return_penalty)
    
    return float(min(adjusted_sharpe, 1_000_000))


def sharpe_ratio_optimization(data):
    """Directly maximize the Adjusted Sharpe Ratio."""
    n_days = len(data)
    
    def objective(positions):
        return -calculate_adjusted_sharpe(data, positions)
    
    best_result = None
    best_score = float('-inf')
    
    initial_guesses = [
        np.full(n_days, 0.5),
        np.full(n_days, 1.0),
        np.random.uniform(0.3, 0.7, n_days),
        np.linspace(0.2, 0.8, n_days),
    ]
    
    bounds = Bounds(MIN_POSITION, MAX_POSITION)
    
    for x0 in initial_guesses:
        try:
            result = minimize(
                objective,
                x0,
                method='SLSQP',
                bounds=bounds,
                options={'maxiter': 1000, 'ftol': 1e-8}
            )
            
            if result.success and -result.fun > best_score:
                best_score = -result.fun
                best_result = result
        except:
            continue
    
    if best_result is not None:
        return np.clip(best_result.x, MIN_POSITION, MAX_POSITION)
    else:
        return np.full(n_days, 0.5)
    

In [8]:
df_tail = df_train_opt.tail(LOOKBACK_WINDOW).reset_index(drop=True)

# 最適化実行
optimized_positions = sharpe_ratio_optimization(df_tail)

# 評価指標を計算
score = calculate_adjusted_sharpe(df_tail, optimized_positions)

# 評価用の詳細を計算
df_eval = df_tail.copy()
df_eval['position'] = optimized_positions
df_eval['strategy_returns'] = (
    df_eval['risk_free_rate'] * (1 - df_eval['position']) +
    df_eval['position'] * df_eval['forward_returns']
)
df_eval['excess_returns'] = df_eval['strategy_returns'] - df_eval['risk_free_rate']

# 結果の集計
mean_position = df_eval['position'].mean()
strategy_volatility = df_eval['strategy_returns'].std() * np.sqrt(TRADING_DAYS_PER_YEAR) * 100
mean_excess_return = df_eval['excess_returns'].mean() * TRADING_DAYS_PER_YEAR * 100

# 出力
print("📈 --- 最適化結果（直近180日） ---")
print(f"調整済みシャープ比 : {score:.4f}")
print(f"平均ポジション       : {mean_position:.4f}")
print(f"戦略ボラティリティ   : {strategy_volatility:.2f}%")
print(f"平均超過リターン     : {mean_excess_return:.4f}%")
print()
print("最初の10日間の最適ポジション:")
print(optimized_positions[:10])

📈 --- 最適化結果（直近180日） ---
調整済みシャープ比 : 17.3963
平均ポジション       : 0.1487
戦略ボラティリティ   : 0.45%
平均超過リターン     : 7.8371%

最初の10日間の最適ポジション:
[9.84723480e-02 5.23570977e-02 2.06594242e-08 3.20459986e-10
 3.53812458e-10 2.50123635e-10 1.67880939e-10 4.64822801e-02
 1.02621864e-01 3.51297301e-10]
